# Use this notebook for developing mlframe prototype 2

## Object-oriented approach. Testing on DL with benchmarks.

In [1]:
import multiprocessing
from Queue import Queue, Empty
import subprocess
import os
import time
import re
import sys

In [2]:
def printObj(obj):
    for key,val in obj.__dict__.iteritems():
        print key,"=",val

In [95]:
# Class for executing local and remote commands in background processes.
# Command must be in multiprocessing.Manager.dict().
# Command must be a string with the command and arguments separated with spaces.
# Should be called from BashExecutor class.
# Usage sample: 
# d = BashExecutor(command,hostname=hostname)
# d.start()
#  command - string representation of the command and arguments.
# exec_remote.sh must output subprocess exit code in the form:
# exitcode=N
# , where N is the number.
class BackgroundExecutor(multiprocessing.Process):
    
    def __init__(self, d = {}, debug=False, hostname=""):
        super(BackgroundExecutor,self).__init__()
        self.debug = debug
        self.hostname = hostname
        self.exitcode_pat = re.compile("^exitcode=(\d+)")
        self.d = d
        self.command = d["command"].split(" ")
        if debug:
            print "In ",self.name," command='",self.command,"'"
        
        
    # Poll exit code of self.proc and store it if not None.
    def poll(self):
        exitcode = self.proc.poll()
        if exitcode is not None:
            self.setExitCode(exitcode)
        return exitcode
    
    # Set given exit code to Command class object.
    # Called from poll() method.
    def setExitCode(self, ec):
#         if self.debug:
#             print "Manager.dict object:",repr(self.d)
#             print type(self.d)
#             printObj(self.d)
        if self.d["exitcode"] == "":
            #if self.debug: print "setting exit code to",ec,
            self.d["exitcode"] = ec                        
        
            
    def run(self):
        if self.debug: 
            print "In {}. Calling {}".format(self.name,self.command)
            
        command = self.command
        proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, shell=False)
        self.proc = proc
        if self.debug: print self.name, "process started"
        std = ""
        for std in iter(proc.stdout.readline, b''):
            if std is not None and len(std) > 0:
                self.d["stdout"] = self.d["stdout"] + std
                if self.debug:
                    self.d[1]=1                    
                    assert self.d[1] ==1, "Cannot set to Manager dictionary"
                print std,
            
            time.sleep(.5)
        print "Exit code:",self.poll()
    
    def nameYourself(self):
        cp = multiprocessing.current_process()
        print "name",cp._name
        print "parent pid",cp._parent_pid
        print "id",cp._identity
    
    def getExitcode(self):
        return self.d["exitcode"]
            

In [96]:
# Class for calling BackgroundExecutor with multiprocessing.Manager.dict object,
# which stores string representation of the command, and after execution: its ouptput and exit code.
class BashExecutor:
    def __init__(self, command, debug=False):
        manager = multiprocessing.Manager()
        self.d = manager.dict()
        self.d["command"] = command
        if debug:
            print "In BashExecutor:"
            print "Command set to '",self.d["command"],"'"
            print "command type:",type(self.d["command"])
        self.d["stdout"] = ""
        self.d["exitcode"] = ""
        self.debug = debug
        self.BE = BackgroundExecutor(self.d, self.debug)
        
    def start(self):
        self.BE.start()
        
    def getExitcode(self):        
        return self.d["exitcode"]
    
    def getStdout(self):
        return self.d["stdout"]
    
    def getCommand(self):
        return self.d["command"]
        
    def __str__(self):
        s = self.d["command"]
        if self.d["exitcode"] != "":
            s += " ("+str(self.d["exitcode"])+")"            
        return s
    
    def __repr__(self):
        s = self.d["command"]
        if self.d["exitcode"] != "":
            s += " ("+str(self.d["exitcode"])+")"            
        return s
        

In [119]:
# Class for storing host-related data: hostname, access key and username.
# Stores commands (instances of Command class) executed on the host.
# Has methods for connecting to the host with ssh, connection test, executing commands.
class Host(object):
    
    def __init__(self, hostname, address="localhost", user="", key="", debug=False, ssh_options="", scp_options=""):
        self.hostname = hostname
        self.address = address
        self.user = user
        key = key.replace("~",os.environ['HOME'])
        self.key = key
        self.debug = debug
        self.ssh_command = "ssh"
        self.scp_command = "scp"
        if key != "":
            self.ssh_command += " -i "+key
            self.scp_command += " -i "+key
        if ssh_options != "":
            self.ssh_command += " -o "+ssh_options
        if scp_options != "":
            self.scp_command += " -o "+scp_options

        self.host=""
        if user != "":
            self.host += user+"@"
        self.host += address
        self.commands = []
        if debug:
            print "hostname,address,user,key:",self.hostname,self.address,self.user,self.key
            print "ssh command:",self.ssh_command.replace(' ','.'),"host:",self.host
        
    def ping(self, N=5):        
        comm = BashExecutor("ping -c "+str(N)+" "+self.address, debug=self.debug)
        #print "Append:",self.commands.append(comm)
        index = len(self.commands)
        self.commands.append(comm)
        comm.start()        
        return index
        
    
    # Execute command on the server
    # If command is a script file, copy the file before executing it.
    def execute(self,command,options=""):
        # Deside if command is a script name or just a command
        package_directory = os.path.dirname(os.getcwd())
        scripts_location=os.path.realpath(os.path.join(package_directory,"mlframe","scripts"))
        command_list = command.split(" ")
        command_file = command_list[0]
        command_script_path = os.path.join(scripts_location, command_file)        
        if os.path.isfile(command_script_path):
            if self.debug: print "script file exists:",command_script_path
            # Joined command: execute remote script file and delete it
            command = "./"+command + " && rm "+command
            if self.copyScriptFile(command_script_path) != 0:
                # Error copying script file
                return -1
        if self.debug: print "Command:",command
        
        if options != "":
            options = " "+options
        command = self.ssh_command+options+" "+self.host+" "+command
        if self.debug:
            print "Executing:",command.replace(' ','.')
        
        comm = BashExecutor(command, debug=self.debug)
        index = len(self.commands)
        self.commands.append(comm)
        comm.start()        
        return index
    
    
    # Copy script file to the host.
    # Called every time before remote script execution.
    def copyScriptFile(self, script_file, options=""):
        command = self.scp_command
        if options != 0:
            command += " "+options
        command += script_file + " "+self.host+":"
        if self.debug: print "Copy command:",command
        proc = subprocess.Popen(command.split(" "), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, shell=False)
        std,stderr = proc.communicate()
        exitcode = proc.poll()
        if exitcode != 0:        
            print "Error copying",script_file,"to",self.hostname,exitcode
            print std
            print "!",stderr
            
        return exitcode
    
    def connect_test(self):
        options = "-o ConnectTimeout=5"
        return self.execute("hostname",options=options)
        

In [120]:
dl_serv = Host("DL","DL",debug=True)

hostname,address,user,key: DL DL  
ssh command: ssh host: DL


In [121]:
dl_serv.execute("test.sh")

script file exists: /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh
Copy command: scp /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh DL:
Command: ./test.sh && rm test.sh
Executing: ssh.DL../test.sh.&&.rm.test.sh
In BashExecutor:
Command set to ' ssh DL ./test.sh && rm test.sh '
command type: <type 'str'>
In  BackgroundExecutor-100  command=' ['ssh', 'DL', './test.sh', '&&', 'rm', 'test.sh'] '


0

In BackgroundExecutor-100. Calling ['ssh', 'DL', './test.sh', '&&', 'rm', 'test.sh']
BackgroundExecutor-100 process started
Running test command on DL-Server pars: 
DL-Server():1
():1
DL-Server() err:1
DL-Server():2
():2
DL-Server():3
():3
DL-Server() err:2
DL-Server() err:3
DL-Server():4
():4
DL-Server():5
():5
DL-Server() err:4
DL-Server() err:5
Exit code: 125


In [122]:
key = "~/.ssh/id_rsa_com"
mouse = Host("mouse","mouse.local",user="peter",key=key,debug=True)

hostname,address,user,key: mouse mouse.local peter /Users/peterbryzgalov/.ssh/id_rsa_com
ssh command: ssh.-i./Users/peterbryzgalov/.ssh/id_rsa_com host: peter@mouse.local


In [123]:
mouse.execute("test.sh")

script file exists: /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh
Copy command: scp -i /Users/peterbryzgalov/.ssh/id_rsa_com /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh peter@mouse.local:
Command: ./test.sh && rm test.sh
Executing: ssh.-i./Users/peterbryzgalov/.ssh/id_rsa_com.peter@mouse.local../test.sh.&&.rm.test.sh
In BashExecutor:
Command set to ' ssh -i /Users/peterbryzgalov/.ssh/id_rsa_com peter@mouse.local ./test.sh && rm test.sh '
command type: <type 'str'>
In  BackgroundExecutor-102  command=' ['ssh', '-i', '/Users/peterbryzgalov/.ssh/id_rsa_com', 'peter@mouse.local', './test.sh', '&&', 'rm', 'test.sh'] '


0

In BackgroundExecutor-102. Calling ['ssh', '-i', '/Users/peterbryzgalov/.ssh/id_rsa_com', 'peter@mouse.local', './test.sh', '&&', 'rm', 'test.sh']
BackgroundExecutor-102 process started
Running test command on mouse pars: 
mouse():1
():1
mouse() err:1
mouse():2
():2
mouse() err:2
mouse():3
():3
mouse():4
():4
mouse() err:3
mouse() err:4
mouse():5
():5
mouse() err:5
Exit code: 125


In [71]:
dl_serv.connect_test()

Test if script file exists in scripts directory /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/hostname
Command: hostname
Executing: ssh.-o.ConnectTimeout=5.DL.hostname
In BashExecutor:
Command set to ' ssh -o ConnectTimeout=5 DL hostname '
command type: <type 'str'>
In  BackgroundExecutor-78  command=' ['ssh', '-o', 'ConnectTimeout=5', 'DL', 'hostname'] '


0

In BackgroundExecutor-78. Calling ['ssh', '-o', 'ConnectTimeout=5', 'DL', 'hostname']
BackgroundExecutor-78 process started
DL-Server
Exit code: setting exit code to 0 0


In [76]:
dl_serv.execute("nvidia-smi dmon -c 2 -s u")

Test if script file exists in scripts directory /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/nvidia-smi
Command: nvidia-smi dmon -c 2 -s u
Executing: ssh.DL.nvidia-smi.dmon.-c.2.-s.u
In BashExecutor:
Command set to ' ssh DL nvidia-smi dmon -c 2 -s u '
command type: <type 'str'>
In  BackgroundExecutor-82  command=' ['ssh', 'DL', 'nvidia-smi', 'dmon', '-c', '2', '-s', 'u'] '


0

In BackgroundExecutor-82. Calling ['ssh', 'DL', 'nvidia-smi', 'dmon', '-c', '2', '-s', 'u']
BackgroundExecutor-82 process started
# gpu    sm   mem   enc   dec
# Idx     %     %     %     %
    0    99    31     0     0
    1    99    32     0     0
    2    99    41     0     0
    3    99    32     0     0
    4    99    30     0     0
    5    99    37     0     0
    6    99    42     0     0
    7    99    33     0     0
    0    99    31     0     0
    1   100    26     0     0
    2    99    35     0     0
    3   100    31     0     0
    4   100    26     0     0
    5    99    28     0     0
    6    99    30     0     0
    7    99    39     0     0
Exit code: setting exit code to 0 0


In [56]:
test = mouse.ping(1)
test = mouse.connect_test()
test = mouse.execute("hostname && date")

PING mouse.local (192.168.83.30): 56 data bytes
64 bytes from 192.168.83.30: icmp_seq=0 ttl=64 time=0.698 ms
mouse
mouse

Fri Mar 23 12:17:31 JST 2018
Exit code: 0
--- mouse.local ping statistics ---
Exit code: 0
1 packets transmitted, 1 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 0.698/0.698/0.698/0.000 ms
Exit code: 0


In [57]:
print test
i=test
print mouse.commands[i].getCommand()
print mouse.commands[i].getStdout()
print mouse.commands[i].getExitcode()

2
ssh -i /Users/peterbryzgalov/.ssh/id_rsa_com peter@mouse.local hostname && date
mouse
Fri Mar 23 12:17:31 JST 2018

0


In [58]:
print mouse.commands

[ping -c 1 mouse.local (0), ssh -i /Users/peterbryzgalov/.ssh/id_rsa_com -o ConnectTimeout=5 peter@mouse.local hostname (0), ssh -i /Users/peterbryzgalov/.ssh/id_rsa_com peter@mouse.local hostname && date (0)]


In [61]:
# Execute command on remote (or local) host in background, using exec_remote.sh and BashExecutor instance.
# Command can be any command or executable file name.
# Arguments must be in a string separated with spaces.
def RemoteExec(host, command_and_args, debug=False):    
    if debug: print "RemoteExec commands and args:",command_and_args
    CA = command_and_args.split(" ")
    if debug: print CA
    command = CA[0]
    args = CA[1:]
    if debug: print "Command:",command
    if debug: print "Args:",args
    package_directory = os.path.dirname(os.getcwd())
    scripts_location=os.path.realpath(os.path.join(package_directory,"mlframe","scripts"))
    exec_remote_script="exec_remote.sh"
    script_path = os.path.join(scripts_location,exec_remote_script)
    
    # Merge args into a string
    command_script_path = os.path.join(scripts_location, command)
    if debug: print "Test if script file exists in RemoteExec",command_script_path
    if os.path.isfile(command_script_path):
        command = command_script_path
    if debug: print "Command:",command
          
    if host == "" or host == "localhost" or host == "127.0.07":
        command_args_list = [command] + args
    else:
        command_args_list = [script_path] + [host] + [command] + args
    if debug: print "Calling BashExecutor with args",command_args_list
    d = BashExecutor(command_args_list, hostname=host, debug=debug)
    d.start()

    #for d in jobs:
    #    d.join()
    
    if debug: print "finished",host,command

In [ ]:
muse = Host("muse","52.158.238.181", user="ubuntu", key=key,debug=False)

In [ ]:
muse.ping()
muse.connect_test()

In [59]:
reedbush = Host("reedbush","reedbush.cc.u-tokyo.ac.jp",key="~/.ssh/id_rsa_com",user="i96005")

In [60]:
reedbush.execute("test.sh")

0

bash: test.sh: command not found
Exit code: 127


In [62]:
RemoteExec(reedbush,"test.sh",debug=True)

RemoteExec commands and args: test.sh
['test.sh']
Command: test.sh
Args: []
Test if script file exists in RemoteExec /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh
Command: /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh
Calling BashExecutor with args ['/Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/exec_remote.sh', <__main__.Host object at 0x1083a9750>, '/Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh']


TypeError: __init__() got an unexpected keyword argument 'hostname'